<a href="https://colab.research.google.com/github/Malleshcr7/AI-ML-Projects/blob/main/Datasetcreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random

tasks = ['Study', 'Sleep', 'Exercise', 'Read', 'Eat', 'Work']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

data = []

for user_id in range(1, 6):  # 5 users
    for day in days:
        for _ in range(3):  # 3 tasks per day
            task = random.choice(tasks)
            duration = random.randint(30, 180)  # minutes
            focus = round(random.uniform(0.5, 1.0), 2)
            start_hour = random.randint(6, 22)
            start_time = f"{start_hour:02d}:{random.randint(0,59):02d}"
            recommended_hour = min(23, max(0, int(start_hour + (focus - 0.75) * 4)))

            data.append({
                "UserID": user_id,
                "Task": task,
                "DayOfWeek": day,
                "Duration": duration,
                "FocusLevel": focus,
                "StartTime": start_time,
                "OutputHour": recommended_hour
            })

df = pd.DataFrame(data)
df.to_csv("daily_timetable_data.csv", index=False)
print("✅ Dataset saved as daily_timetable_data.csv")


✅ Dataset saved as daily_timetable_data.csv


In [ ]:
from google.colab import files
files.download("daily_timetable_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib

# Load dataset
df = pd.read_csv("daily_timetable_data.csv")

# Extract StartHour from StartTime
df['StartHour'] = pd.to_datetime(df['StartTime'], format='%H:%M').dt.hour

# One-hot encode categorical features
df_encoded = pd.get_dummies(df[['Task', 'DayOfWeek']])

# Final feature set
X = pd.concat([df_encoded, df[['Duration', 'FocusLevel', 'StartHour']]], axis=1)
y = df['OutputHour']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"📊 Mean Absolute Error: {mae:.2f} hours")


📊 Mean Absolute Error: 0.41 hours


In [ ]:
# Save model
joblib.dump(model, "timetable_model.pkl")
print("✅ Model saved as timetable_model.pkl")

# Download the model to your system
from google.colab import files
files.download("timetable_model.pkl")


✅ Model saved as timetable_model.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def predict_best_time(task, day, duration, focus, start_hour):
    sample = {
        "Task": task,
        "DayOfWeek": day,
        "Duration": duration,
        "FocusLevel": focus,
        "StartHour": start_hour
    }

    # Columns from your training dataset
    task_cols = ['Task_Eat', 'Task_Exercise', 'Task_Read', 'Task_Sleep', 'Task_Study', 'Task_Work']
    day_cols = ['DayOfWeek_Friday', 'DayOfWeek_Monday', 'DayOfWeek_Saturday', 'DayOfWeek_Sunday',
                'DayOfWeek_Thursday', 'DayOfWeek_Tuesday', 'DayOfWeek_Wednesday']

    features = dict.fromkeys(task_cols + day_cols + ['Duration', 'FocusLevel', 'StartHour'], 0)
    features[f'Task_{sample["Task"]}'] = 1
    features[f'DayOfWeek_{sample["DayOfWeek"]}'] = 1
    features['Duration'] = sample['Duration']
    features['FocusLevel'] = sample['FocusLevel']
    features['StartHour'] = sample['StartHour']

    input_df = pd.DataFrame([features])
    predicted_hour = model.predict(input_df)[0]
    return round(predicted_hour)
    print("📚 Study:", predict_best_time("Study", "Monday", 120, 0.9, 8))
